<a href="https://colab.research.google.com/github/KevinWang676/Talking-Face-GPT/blob/main/%E9%9A%8F%E5%8F%AB%E9%9A%8F%E5%88%B0%E7%9A%84AI%E5%8A%A9%E6%89%8B(%E8%A7%86%E9%A2%91%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 KB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 487.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.3 MB/s eta 0:00:00


In [3]:
import whisper

model = whisper.load_model("base")


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 58.1MiB/s]


In [4]:
#! pip install gradio -q

In [5]:
!git lfs install
%cd /content
!git clone https://huggingface.co/camenduru/pocketsphinx-20.04-t4 pocketsphinx
%cd  /content/pocketsphinx
!sudo cmake --build build --target install
%cd /content
!git clone https://huggingface.co/camenduru/one-shot-talking-face-20.04-t4 one-shot-talking-face
%cd /content/one-shot-talking-face
!pip install -r /content/one-shot-talking-face/requirements.txt
!chmod 755 /content/one-shot-talking-face/OpenFace/FeatureExtraction
!mkdir /content/train
!apt install -qq libgtk2.0-0 jq -y
!pip install -q imageio-ffmpeg gradio

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
/content
Cloning into 'pocketsphinx'...
remote: Enumerating objects: 1658, done.
remote: Total 1658 (delta 0), reused 0 (delta 0), pack-reused 1658
Receiving objects: 100% (1658/1658), 8.51 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (592/592), done.
Filtering content: 100% (20/20), 34.59 MiB | 14.45 MiB/s, done.
/content/pocketsphinx
-- SIZEOF_LONG="8" ; SIZEOF_LONG_LONG="8"
-- WORDS_BIGENDIAN="0"
-- SPHINX_DEBUG="" ; DEBUG_ENDIAN=""
-- FIXED_POINT="OFF" ; DEFAULT_RADIX="12"
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Checking for module 'libpulse-simple'
--   No package 'libpulse-simple' found
-- Checking for module 'portaudio-2.0'
--   No package 'portaudio-2.0' found
-- MODELDIR="/usr/local/share/pocketsphinx/model"
-- Configuring done
-- Generating done
-- Build files have been written to: /content/pocketsphinx/build
Scanning dependencies of target pocketsphinx
[  0%] Building 

In [6]:
pip install gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from gtts import gTTS

In [8]:
#tts = gTTS('Hello there, nice to meet you!', lang='en', tld='us')
#tts.save('1.wav')
#sound_file = '1.wav'
#Audio(sound_file, autoplay=True)

In [9]:
import gradio as gr

In [10]:
import openai
import os, subprocess, torchaudio
from PIL import Image

mes1 = [
    {"role": "system", "content": "You are a TOEFL examiner. Help me improve my oral Englsih and give me feedback. Your response must be less than 30 words."}
]

mes2 = [
    {"role": "system", "content": "You are a mental health therapist. Your name is Tina. Your response must be less than 30 words."}
]

mes3 = [
    {"role": "system", "content": "You are my personal assistant. Your name is Alice. Your response must be less than 30 words."}
]

res = []

question = []

def transcribe(apikey, image_in, audio, choice1, choice2):

    openai.api_key = apikey
    
    # time.sleep(3)
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    res.append(result.text)

    if choice1 == "TOEFL":
      messages = mes1
    elif choice1 == "Therapist":
      messages = mes2
    elif choice1 == "Alice":
      messages = mes3

    # chatgpt
    n = len(res)
    content = res[n-1]
    messages.append({"role": "user", "content": content})

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = messages
    )

    chat_response = completion.choices[0].message.content

    messages.append({"role": "assistant", "content": chat_response})   

    if choice2 == "Chinese":
      tts = gTTS(chat_response, lang='zh-CN')
    elif choice2 == "English":
      tts = gTTS(chat_response, lang='en', tld='us')

    tts.save('1.wav')
    audio_in = '1.wav'

    waveform, sample_rate = torchaudio.load(audio_in)
    torchaudio.save("/content/audio.wav", waveform, sample_rate, encoding="PCM_S", bits_per_sample=16)
    image = Image.open(image_in)
    image.save("/content/image.png")

    pocketsphinx_run = subprocess.run(['pocketsphinx', '-phone_align', 'yes', 'single', '/content/audio.wav'], check=True, capture_output=True)
    jq_run = subprocess.run(['jq', '[.w[]|{word: (.t | ascii_upcase | sub("<S>"; "sil") | sub("<SIL>"; "sil") | sub("\\\(2\\\)"; "") | sub("\\\(3\\\)"; "") | sub("\\\(4\\\)"; "") | sub("\\\[SPEECH\\\]"; "SIL") | sub("\\\[NOISE\\\]"; "SIL")), phones: [.w[]|{ph: .t | sub("\\\+SPN\\\+"; "SIL") | sub("\\\+NSN\\\+"; "SIL"), bg: (.b*100)|floor, ed: (.b*100+.d*100)|floor}]}]'], input=pocketsphinx_run.stdout, capture_output=True)
    with open("/content/test.json", "w") as f:
        f.write(jq_run.stdout.decode('utf-8').strip())

    os.system(f"cd /content/one-shot-talking-face && python3 -B test_script.py --img_path /content/image.png --audio_path /content/audio.wav --phoneme_path /content/test.json --save_dir /content/train")
    
    question.append(chat_response)

    return [result.text, chat_response, "/content/train/image_audio.mp4", res, question]

output_1 = gr.Textbox(label="Speech to Text")
output_2 = gr.Textbox(label="ChatGPT Output")
output_3 = gr.Video(label="Vedio")

gr.Interface(
    title = '🥳💬💕 - TalktoAI，随时随地，谈天说地！', 
    theme="huggingface",
    description = "🤖 - 让有人文关怀的AI造福每一个人！AI向善，文明璀璨！Enable the future！",
    fn=transcribe, 
    inputs=[
        gr.Textbox(lines=1, label = "请填写您的OpenAI_API_key"),
        gr.Image(label = "请上传图片", type="filepath"),
        gr.inputs.Audio(source="microphone", type="filepath"),
        gr.Radio(["TOEFL", "Therapist", "Alice"], label="TOEFL Examiner, Therapist Tina, or Assistant Alice?"),
        gr.Radio(["Chinese", "English"], label="Chinese or English?")
    ],
    outputs=[
      output_1, output_2, output_3
    ],
    ).launch()

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:319: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:491: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [11]:
res1 = ''
for i in range(0, len(res)):
  res1 = res1 + " \n\n " + res[i]
question1 = ''
for i in range(0, len(question)):
  question1 = question1 + " \n\n " + question[i]
print(f'Your Response: {res1}')
print('')
print('')
print('-------------------------TalktoAI，随时随地，谈天说地！-------------------------')
print('')
print(f'Question: {question1}')

Your Response: 


-------------------------TalktoAI，随时随地，谈天说地！-------------------------

Question: 
